In [ ]:
import os

model_save_path = "saved_model_cnn1"
cols = ["KOSPI", "KOSPI_START", "KOSPI_HIGH", "KOSPI_LOW", "NASDAQ", "DOW", "CR", "GOLD"]
len_x_ARMA = 20
len_y_nextday = 20
scale_method = "minmax"

In [ ]:
from kospi_predict import Crawler

crawler = Crawler(crawl_page_max=30, perPage=100)
# crawler.crawlData(cols, save=True)
crawler.loadFromSavedFile(cols)
df_crawled = crawler.removeNan()
df_crawled = df_crawled.loc[:2000]
df_crawled

In [ ]:
from kospi_predict import DataPreprocessor

dpp = DataPreprocessor(df_crawled, cols, scale_method, model_save_path)
dpp.sortByDate()
dpp.makeDiffRatio()
dpp.scalingForModeling()
dpp.makeAR(0, len_x_ARMA)
dpp.makeMA(2, len_x_ARMA)
dpp.makeTargetYs(len_y_nextday)
dpp.cutoffData(len_x_ARMA, len_y_nextday)

In [129]:
dpp.df

,date,KOSPI,KOSPI_START,KOSPI_HIGH,KOSPI_LOW,NASDAQ,DOW,CR,GOLD,X_KOSPI_DIFF_RATIO,...,Y_KOSPI_nextday_11,Y_KOSPI_nextday_12,Y_KOSPI_nextday_13,Y_KOSPI_nextday_14,Y_KOSPI_nextday_15,Y_KOSPI_nextday_16,Y_KOSPI_nextday_17,Y_KOSPI_nextday_18,Y_KOSPI_nextday_19,Y_KOSPI_nextday_20
0,2014-02-26,0.277732,0.267066,0.254056,0.278280,0.024098,0.037656,0.213971,0.289114,1.003008,...,0.258036,0.250199,0.254329,0.261192,0.259822,0.249993,0.258339,0.264082,0.261755,0.274236
1,2014-02-27,0.281878,0.270561,0.257663,0.283549,0.024469,0.038532,0.218750,0.272687,1.003887,...,0.250199,0.254329,0.261192,0.259822,0.249993,0.258339,0.264082,0.261755,0.274236,0.281629
2,2014-02-28,0.282723,0.274182,0.258096,0.287294,0.026697,0.041996,0.219485,0.273884,1.000789,...,0.254329,0.261192,0.259822,0.249993,0.258339,0.264082,0.261755,0.274236,0.281629,0.283269
3,2014-03-03,0.274442,0.269496,0.251432,0.277843,0.025800,0.044286,0.231618,0.280485,0.992273,...,0.261192,0.259822,0.249993,0.258339,0.264082,0.261755,0.274236,0.281629,0.283269,0.285765
4,2014-03-04,0.268715,0.262221,0.248320,0.276496,0.023245,0.037114,0.233456,0.287086,0.994615,...,0.259822,0.249993,0.258339,0.264082,0.261755,0.274236,0.281629,0.283269,0.285765,0.289212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,2022-02-09,0.709694,0.708997,0.698388,0.708013,0.845530,0.937608,0.691176,0.768085,1.008149,...,0.644717,0.659851,0.671985,0.674334,0.697911,0.679698,0.646076,0.630428,0.661777,0.651472
1956,2022-02-10,0.711361,0.717085,0.706663,0.712458,0.870066,0.951855,0.693015,0.773577,1.001112,...,0.659851,0.671985,0.674334,0.697911,0.679698,0.646076,0.630428,0.661777,0.651472,0.643012
1957,2022-02-11,0.698252,0.690706,0.694675,0.698084,0.844799,0.927285,0.694853,0.769025,0.991262,...,0.671985,0.674334,0.697911,0.679698,0.646076,0.630428,0.661777,0.651472,0.643012,0.629957
1958,2022-02-14,0.674854,0.677577,0.671344,0.672847,0.812090,0.903785,0.696691,0.798367,0.984267,...,0.674334,0.697911,0.679698,0.646076,0.630428,0.661777,0.651472,0.643012,0.629957,0.650362


In [132]:
import cv2
import numpy as np

def makeImage2D(df):
    #행 : X_*_AR, X_*_DIFF_RATIO_AR, X_*_MA, X_*_DIFF_RATIO_MA  *  원천 X 갯수(8개라면 8*4 = 32행)
    #열 : ARMA 갯수. 20이라면 20 = 20열
    col_made_list = ["X_*_AR", "X_*_DIFF_RATIO_AR", "X_*_MA"]
    img_list = []
    label_list = []
    date_list = []
    y_list = []
    for item in df.iterrows():
        this_mat_list = []
        for col in cols: #8개 ["KOSPI", "KOSPI_START", "KOSPI_HIGH", "KOSPI_LOW", "NASDAQ", "DOW", "CR", "GOLD"]
            for col_made in col_made_list: #4개 ["X_*_AR", "X_*_DIFF_RATIO_AR", "X_*_MA", "X_*_DIFF_RATIO_MA"]
                this_col = col_made.replace("*", col) #X_KOSPI_AR
                this_row_list = []
                for i in range(0, len_x_ARMA): #20개. 0~20
                    if "MA" in this_col:
                        i += 2
                    this_col_made = "{}{}".format(this_col, i) #X_KOSPI_AR0
                    this_row_list.append([df.loc[item[0], this_col_made]])
                this_mat_list.append(this_row_list)
        this_mat = np.array(this_mat_list)
        this_mat = this_mat
        img_list.append(this_mat)
        cv2.imwrite("data_cnn/{}.png".format(df.loc[item[0], "date"]), this_mat * 255)

        #라벨 체크
        this_label = np.nan
        if item[0]+1 < len(df): #0 1 2 ... 29 / 30
            this_label = 0
            next_day_diff = df.loc[item[0]+1, "KOSPI"] - df.loc[item[0], "KOSPI"] #다음 날 종가에서 오늘 종가를 뺀값
            if next_day_diff > 0:
                this_label = 1
        label_list.append(this_label)

        #날짜 리스트 입력
        date_list.append(df.loc[item[0], "date"])

        #y_list 입력
        this_y_list = []
        for y in [x for x in df.columns.to_list() if "Y_" in x]:
            this_y_list.append(df.loc[item[0], y])
        y_list.append(this_y_list)

    img_list = np.array(img_list)
    label_list = np.array(label_list)
    date_list = np.array(date_list)
    y_list = np.array(y_list)
    df_result = pd.DataFrame({"date" : date_list, "label" : label_list})

    return img_list, label_list, date_list, y_list, df_result

img_list, label_list, date_list, y_list, _ = makeImage2D(dpp.df)
len(img_list), len(label_list), len(date_list), len(y_list)

(1959, 1960, 1959, 1959)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(img_list, label_list, test_size=0.2, shuffle=False, random_state=8699)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(24, 20, 1)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  # tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(2, activation="softmax")
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()
epochs=200
history = model.fit(
  x_train,
  y_train,
  epochs=epochs,
  validation_split = 0.1,
  verbose = 1
)

In [ ]:
import pandas as pd
df_history = pd.DataFrame(history.history)
df_history

In [ ]:
preds = model.predict(x_test)
test_results = []
for idx, pred in enumerate(preds):
    this_label = y_test[idx]
    this_predict = np.argmax(pred)
    acc = 0
    if this_label == this_predict:
        acc = 1
    test_results.append({"label" : this_label, "predict" : this_predict, "acc" : acc})

test_results = pd.DataFrame(test_results)
test_results

In [ ]:
acc_ratio = test_results["acc"].sum() / len(test_results)
print("테스트 정확도 : {}/{} ({:.2f})".format(test_results["acc"].sum(), len(test_results), acc_ratio))

최근 날짜로 테스트

In [ ]:
crawler_pred = Crawler(crawl_page_max=1, perPage=100) #이전 추세도 볼겸 넉넉히 수집
crawler_pred.crawlData(cols)
df_crawled_pred = crawler_pred.removeNan()
df_crawled_pred

In [ ]:
#아래는 예측을 위한 전처리
dpp_pred = DataPreprocessor(df_crawled_pred.loc[:100], cols, scale_method, model_save_path)
dpp_pred.sortByDate()
dpp_pred.makeDiffRatio()
dpp_pred.scalingForPredict()
dpp_pred.makeAR(0, len_x_ARMA)
dpp_pred.makeMA(2, len_x_ARMA)
dpp_pred.cutoffData(len_x_ARMA, 0)

In [126]:
dpp_pred.df

,date,KOSPI,KOSPI_START,KOSPI_HIGH,KOSPI_LOW,NASDAQ,DOW,CR,GOLD,X_KOSPI_DIFF_RATIO,...,X_GOLD_MA17,X_GOLD_DIFF_RATIO_MA17,X_GOLD_MA18,X_GOLD_DIFF_RATIO_MA18,X_GOLD_MA19,X_GOLD_DIFF_RATIO_MA19,X_GOLD_MA20,X_GOLD_DIFF_RATIO_MA20,X_GOLD_MA21,X_GOLD_DIFF_RATIO_MA21
0,2021-11-30,0.747669,0.796424,0.792617,0.745308,0.977231,0.922354,0.669118,0.735032,0.975833,...,0.771851,1.000754,0.768892,1.000549,0.766516,1.000134,0.765029,1.000426,0.763182,0.952786
1,2021-12-01,0.780528,0.756779,0.771948,0.753013,0.956905,0.891915,0.634191,0.720584,1.021384,...,0.771644,0.999922,0.769805,1.000557,0.767110,1.000373,0.764942,0.999988,0.763600,1.000273
2,2021-12-02,0.805182,0.764709,0.793918,0.773277,0.933387,0.870368,0.621324,0.712667,1.015708,...,0.769642,0.998911,0.768808,0.999474,0.767215,1.000099,0.764783,0.999947,0.762829,0.999601
3,2021-12-03,0.817663,0.798182,0.810685,0.801784,0.943939,0.899198,0.643382,0.711945,1.007830,...,0.766501,0.998271,0.766476,0.998721,0.765853,0.999265,0.764487,0.999869,0.762302,0.999736
4,2021-12-06,0.820326,0.808499,0.815165,0.804446,0.919409,0.896412,0.639706,0.723949,1.001657,...,0.763245,0.998207,0.763470,0.998344,0.763606,0.998767,0.763157,0.999281,0.761985,0.999856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,2022-03-14,0.643012,0.644715,0.636354,0.643251,0.733540,0.820064,0.849265,0.909579,0.994127,...,0.866607,1.004241,0.862816,1.004907,0.857880,1.004516,0.853665,1.004443,0.849589,1.004422
69,2022-03-15,0.629957,0.631269,0.625355,0.633429,0.711768,0.820113,0.873162,0.875260,0.990883,...,0.872778,1.003341,0.868994,1.003346,0.865277,1.004024,0.860465,1.003697,0.856327,1.003666
70,2022-03-16,0.650362,0.641941,0.635320,0.641484,0.742231,0.848073,0.832721,0.859179,1.014381,...,0.877304,1.002507,0.872916,1.002177,0.869324,1.002243,0.865776,1.002942,0.861169,1.002682
71,2022-03-17,0.669458,0.671116,0.665387,0.676225,0.782688,0.872284,0.753676,0.880455,1.013267,...,0.879187,1.001090,0.876297,1.001901,0.872193,1.001620,0.868817,1.001711,0.865462,1.002402


In [127]:
img_pred_list, label_pred_list, date_list, _ = makeImage2D(dpp_pred.df, cut_latest_data=False)
len(img_pred_list), len(label_pred_list), len(date_list)

(73, 72, 73)

In [128]:
label_pred_list

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1])

In [ ]:
before_day = 1
img_pred = np.array([img_pred_list[-before_day]])
pred = model.predict(img_pred)
test_results = []
predict = np.argmax(pred)
print("{} predict : {}, real : {}".format(date_list[-before_day], predict, label_pred_list[-before_day]))

# CNN + 예측

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(img_list, y_list, test_size=0.2, shuffle=False, random_state=8699)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(24, 20, 1)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(20, activation="relu")
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse',
            optimizer=optimizer,
            metrics=['mae', 'mse'])

model.summary()
epochs=5000
history = model.fit(
  x_train,
  y_train,
  epochs=epochs,
  validation_split = 0.1,
  verbose = 1
)